# `ipytest` Summary

`ipytest` aims to make testing code in IPython notebooks easy. At its core, it offers a way to run pytest tests inside the notebook environment. It is also designed to make the transfer of the tests into proper python modules easy by supporting to use standard `pytest` features.

To get started install `ipytest` via:

```bash
pip install -U ipytest
```

To use `ipytest`, import it and configure the notebook. In most cases, running `ipytest.autoconfig()` will result in reasonable defaults:

- Tests can be executed with the `%%ipytest` magic
- The `pytest` assert rewriting system to get nice assert messages will integrated into the notebook 

For more control, pass the relevant arguments to `ipytest.autconfig()`. For details, see the documentation in the readme.

In [1]:
import ipytest

ipytest.autoconfig()

## Execute tests

To execute test, just decorate the cells containing the tests with the `%%ipytest` magic:

In [2]:
%%ipytest

# define the tests

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2
    
    
def my_func(x):
    return x // 2 * 2 

.                                                                                            [100%]
1 passed in 0.03s


## Using pytest fixtures

Common pytest features, such as fixtures and parametrize, are supported out of the box:

In [3]:
%%ipytest

import pytest

@pytest.mark.parametrize('input,expected', [
    (0, 0),
    (1, 0),
    (2, 2),
    (3, 2),
])
def test_parametrized(input, expected):
    assert my_func(input) == expected
    
    
@pytest.fixture
def my_fixture():
    return 42
    
    
def test_fixture(my_fixture):
    assert my_fixture == 42

.....                                                                                        [100%]
5 passed in 0.03s


## Selecting tests

[Pytest](https://pytest.org) offers a [extensive options](https://docs.pytest.org/en/latest/how-to/usage.html#specifying-which-tests-to-run) to select subsets of tests to run. They can be also used in conjunction with `ipytest`:

In [4]:
%%ipytest -k feature1

def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert True
    
def test_feature2_test1():
    assert False
    
def test_feature2_test2():
    assert False

..                                                                                           [100%]
2 passed, 2 deselected in 0.03s


Tests can also be selected based on node ids. The notebook can be referenced via the special `{MODULE}` name. For example `{MODULE}::test_feature1_test1` references the corresponding test defined in the notebook: 

In [5]:
%%ipytest {MODULE}::test_feature1_test1
        
def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert False

.                                                                                            [100%]
1 passed in 0.03s


[Deselection](https://docs.pytest.org/en/7.1.x/example/pythoncollection.html#deselect-tests-during-test-collection) works as well:

In [6]:
%%ipytest --deselect {MODULE}::test_feature1_test2

def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert False

.                                                                                            [100%]
1 passed, 1 deselected in 0.02s


## Debugging failed tests

The [debugging functionality of pytest](https://docs.pytest.org/en/latest/how-to/failures.html) can be used as well. For example, to debug the first failed test (and then stop the pytest run) use:

In [7]:
%%ipytest -x --pdb

def test_example():
    for i in range(10):
        if i == 5: 
            raise ValueError(i)

F
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> traceback >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

    def test_example():
        for i in range(10):
            if i == 5:
>               raise ValueError(i)
E               ValueError: 5

/tmp/ipykernel_10676/636047835.py:4: ValueError
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> entering PDB >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>> PDB post_mortem (IO-capturing turned off) >>>>>>>>>>>>>>>>>>>>>>>>>>>>>
> /tmp/ipykernel_10676/636047835.py(4)test_example()
      1 def test_example():
      2     for i in range(10):
      3         if i == 5:
----> 4             raise ValueError(i)



ipdb>  p i


5


ipdb>  q



===================================== short test summary info ======================================
FAILED tmp5v4c9y11.py::test_example - ValueError: 5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! stopping after 1 failures !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!! _pytest.outcomes.Exit: Quitting debugger !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
1 failed in 3.38s


## Checking notebooks automatically

`ipytest` itself does not support validating notebooks in a programmatic fashion. For this task, the [`nbval` package](https://nbval.readthedocs.io/en/latest) can be used. In its default configuration, `ipytest` will not raise lead to execution failures, but only display the exception. While this behavior is helpful during interactive development, it will prevent `nbval` from catching errors. To raise errors if any test fails, configure `raise_on_error=True`:

In [ ]:
%%ipytest
# ipytest: raise_on_error=True

def test():
    raise ValueError()